In [ ]:
%matplotlib inline

import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt
import numpy as np

First, let's plot a map of Kilauea and rift zone, and also the location of Pu'u O'o vent and the first fissure appearance during the 2018 eruption.

The Pu'u O'o vent location is also used to set the map boundary. 

In [ ]:
# location of Pu'u O'o vent
puu_oo_lat = 19.386389
puu_oo_lon = -155.1050

# location of first fissure eruption
puna_eruption_lat = 19.463 
puna_eruption_lon = -154.899

Now run the following cell to plot the map.
Can you recognize the location of the caldera and various vents along the rift zone?

In [ ]:
# plot
fig = plt.figure(dpi=150)

#plot terrain
stamen_terrain = cimgt.Stamen('terrain-background')
projection=stamen_terrain.crs # Set up a custom projection
ax = fig.add_subplot(111,projection=projection)
plot_extent = (puu_oo_lon-0.3, puu_oo_lon+0.3, puu_oo_lat-0.2, puu_oo_lat+0.15) #map boundary set by puu_oo_lat
ax.set_extent(plot_extent)
ax.add_image(stamen_terrain, 12)

gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = None;  gl.ylabels_right = None
ax.set_title(' Kilauea ')

# plot location of Pu'u O'o vent
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(puu_oo_lon), np.array(puu_oo_lat)).T
ax.scatter(x, y, 30, color="b", marker="o", edgecolor="k", zorder=3)
plt.text(x, y, 'vent', va="top", family="monospace", fontsize='small')
    
# plot location of 2018 Lower puna eruption
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(puna_eruption_lon), np.array(puna_eruption_lat)).T
ax.scatter(x, y, 30, color="y", marker="s", edgecolor="k", zorder=3)
plt.text(x, y, 'eruption', va="top", family="monospace", fontsize='small')   


plt.show()

Now we will get an inventory of the seismic stations operated by the Hawaiian Volcano Observatory (network code "HV") using the 'get_stations' function.

We will specifically plot seismometers that has a EHZ channel. EHZ is a channel name for the short period vertical component of the seismometer, which is more sensitive to the weak seismic events. 

In [ ]:
from obspy.clients.fdsn import Client

client = Client("IRIS") # we use IRIS as IRIS holds the seismic data and metadata (e.g., inventory info) 

network="HV"  # network code for Hawaiian Volcano Observatory
station="*"   # * means include all stations
channel="EHZ" # component of the seismometer.  

inv = client.get_stations(network=network,station=station,location="",channel=channel,level="channel")
print(inv)

Run the next cell to make a new map, which includes the seismic stations you just found.

In [ ]:
fig = plt.figure(dpi=150)

#plot map

stamen_terrain = cimgt.Stamen('terrain-background')
projection=stamen_terrain.crs # Set up a custom projection
ax = fig.add_subplot(111,projection=projection)
plot_extent = (puu_oo_lon-0.3, puu_oo_lon+0.3, puu_oo_lat-0.2, puu_oo_lat+0.15)
ax.set_extent(plot_extent)
ax.add_image(stamen_terrain, 12)

gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = None;  gl.ylabels_right = None
ax.set_title(' HV stations ')
 
# Pu'u O'o vent
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(puu_oo_lon), np.array(puu_oo_lat)).T
ax.scatter(x, y, 30, color="b", marker="o", edgecolor="k", zorder=3)
plt.text(x, y, 'vent', va="top", family="monospace", fontsize='small')
    
# 2018 Lower puna eruption
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(puna_eruption_lon), np.array(puna_eruption_lat)).T
ax.scatter(x, y, 30, color="y", marker="s", edgecolor="k", zorder=3)
plt.text(x, y, 'eruption', va="top", family="monospace", fontsize='small')   

# HV stations
stations = []; stla = []; stlo = []
for sta in inv.networks[0].stations:
    #print(sta.code)
    stations.append(sta.code)
    stla.append(sta.latitude)
    stlo.append(sta.longitude)
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(stlo), np.array(stla)).T
ax.scatter(x, y, 30, color="r", marker="v", zorder=3)
for i in range(len(stations)):
    t = plt.text(x[i], y[i], stations[i], va="top", family="monospace",fontsize='xx-small',clip_on=True)
    t.clipbox = ax.bbox

plt.show()




Let's refine the above map, to limit to stations that are running during the eruption.
This is when we need to use UTCDateTime, which is how the obspy script reads date and time. Seismogram recordings and event catalogues are always in UTC (universal) time, so we will use UTC time.  

In [ ]:
from obspy import UTCDateTime

operating_starttime = UTCDateTime("2018-04-30T0:00:00") 

inv = client.get_stations(network=network,station="*",location="",channel=channel,level="channel",
                         starttime = operating_starttime)
print(inv)

In [ ]:
fig = plt.figure(dpi=150)

#plot map

stamen_terrain = cimgt.Stamen('terrain-background')
projection=stamen_terrain.crs # Set up a custom projection
ax = fig.add_subplot(111,projection=projection)
plot_extent = (puu_oo_lon-0.3, puu_oo_lon+0.3, puu_oo_lat-0.2, puu_oo_lat+0.15)
ax.set_extent(plot_extent)
ax.add_image(stamen_terrain, 12)

gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = None;  gl.ylabels_right = None
ax.set_title(' HV stations operating during the 2018 Eruption')
 
# Pu'u O'o vent
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(puu_oo_lon), np.array(puu_oo_lat)).T
ax.scatter(x, y, 30, color="b", marker="o", edgecolor="k", zorder=3)
plt.text(x, y, 'vent', va="top", family="monospace", fontsize='small')
    
# 2018 Lower puna eruption
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(puna_eruption_lon), np.array(puna_eruption_lat)).T
ax.scatter(x, y, 30, color="y", marker="s", edgecolor="k", zorder=3)
plt.text(x, y, 'eruption', va="top", family="monospace", fontsize='small')   

# HV stations
stations = []; stla = []; stlo = []
for sta in inv.networks[0].stations:
    #print(sta.code)
    stations.append(sta.code)
    stla.append(sta.latitude)
    stlo.append(sta.longitude)
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(stlo), np.array(stla)).T
ax.scatter(x, y, 30, color="r", marker="v", zorder=3)
for i in range(len(stations)):
    t = plt.text(x[i], y[i], stations[i], va="top", family="monospace",fontsize='xx-small',clip_on=True)
    t.clipbox = ax.bbox

plt.show()

We will now download a day-long time series from the station closest to the vent using the 'get_waveforms' function. 

We will need two inputs:
- station name (4 digit code)
- a start time (for the time series). Pick a time ~12 hours before the first major activity at the vent (in UTC!)

If you get error messages saying FDSNNNoDataException, that means those stations are not operating (hence no data). 

In [ ]:
stationIN = 'KUPD'  # <--- input station name

starttime = UTCDateTime("2016-05-10T13:00:00")  # <---- adjust start time!
endtime = starttime + 24 * 60 * 60  

stream = client.get_waveforms(network='HV',station = stationIN,location='',channel='EHZ',starttime=starttime, endtime=endtime)
print(stream)


There are many ways to plot the seismogram. One easy way to look a day-long time series is plotting a "day plot" using the obpys function. Run the following cell to see the plot:

In [ ]:
stream.plot(type='dayplot',interval=60)

Combined with recordings from other stations, seismologists are able to detect many seismic activities and put together an event catalog detailing their magnitude, origin time, and location. 

We will need two inputs:
- start time of the catalog
- end time of the catalog

In [ ]:
# start time and end time of the event catalog
starttime = UTCDateTime("2018-07-29T0:00:00") # <--- pick time a day before Puuoo collapse
endtime = UTCDateTime("2018-010-04T0:00:00") # <-- pick time a day after the first fissure appears

# limit events occuring within the map boundary
minlongitude = puu_oo_lon-0.3
maxlongitude = puu_oo_lon+0.3
minlatitude = puu_oo_lat-0.2
maxlatitude = puu_oo_lat+0.15

client_catalog = Client("USGS")  # we switched client to USGS because we will use the catalog compiled by USGS.
cat = client_catalog.get_events(starttime = starttime, endtime = endtime, 
                                      minlatitude = minlatitude, maxlatitude = maxlatitude,
                                      minlongitude = minlongitude, maxlongitude = maxlongitude)
print(cat.__str__(print_all=True))

We will make a final plot showing the events on the map (colored by time). Run the following cell:

In [ ]:
from matplotlib import cm
from matplotlib.dates import DateFormatter,DayLocator

fig = plt.figure(dpi=150)

#plot background map
stamen_terrain = cimgt.Stamen('terrain-background')
projection=stamen_terrain.crs 
ax = fig.add_subplot(111,projection=projection)
plot_extent = (puu_oo_lon-0.3, puu_oo_lon+0.3, puu_oo_lat-0.2, puu_oo_lat+0.15)
ax.set_extent(plot_extent)
ax.add_image(stamen_terrain, 10)
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = None;  gl.ylabels_right = None

# set title
ax.set_title(' Seismicity between %s and %s ' %(starttime.strftime('%b %d'), endtime.strftime('%b %d')))
 
# plot locatino of Pu'u O'o vent
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(puu_oo_lon), np.array(puu_oo_lat)).T
ax.scatter(x, y, 30, color="b", marker="o", edgecolor="k", zorder=3)
plt.text(x, y, 'vent', va="top", family="monospace", fontsize='small')
    
# plot location of 2018 Lower puna eruption
x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(puna_eruption_lon), np.array(puna_eruption_lat)).T
ax.scatter(x, y, 30, color="y", marker="s", edgecolor="k", zorder=3)
plt.text(x, y, 'eruption', va="top", family="monospace", fontsize='small')   

# plot location of seismic events
otime_cb = []; evlat = []; evlon = []; evmag = []
for event in cat:
    otime_cb.append(event.origins[0].time.matplotlib_date)
    evlat.append(event.origins[0].latitude)
    evlon.append(event.origins[0].longitude)
    evmag.append(event.magnitudes[0].mag)

total_event = len(cat)
colormap = cm.get_cmap('magma',total_event)

x, y, _ = projection.transform_points(ccrs.Geodetic(), np.array(evlon), np.array(evlat)).T
im = ax.scatter(x, y, s=evmag, c=otime_cb, cmap=colormap, marker="o", edgecolor=None, zorder=3)
cb = plt.colorbar(im,orientation='horizontal',shrink=0.7,
                  ticks=DayLocator(interval=1),
                  format=DateFormatter('%b %d'))

plt.show()